# 🎬 SAM3 视频目标追踪 (Google Colab)

本 Notebook 使用 SAM3 在 GPU 上进行视频目标追踪，支持：
- 文本提示检测任意目标
- 跨帧目标追踪（ID 保持）
- 批量处理多个视频

**运行前请确保**：
1. 选择 GPU 运行时：`运行时 -> 更改运行时类型 -> T4 GPU`
2. 已登录 Hugging Face 并有 SAM3 访问权限


## 1️⃣ 检查 GPU 并安装依赖


In [ ]:
# 检查 GPU
!nvidia-smi

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# 安装 SAM3 和依赖
!pip install git+https://github.com/facebookresearch/sam3.git
!pip install opencv-python matplotlib scikit-learn tqdm


In [ ]:
# 登录 Hugging Face（需要 SAM3 访问权限）
from huggingface_hub import login

# 方法1：交互式登录
login()

# 方法2：使用 token（取消注释并填入你的 token）
# login(token="your_hf_token_here")


## 2️⃣ 挂载 Google Drive 并上传视频


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 创建工作目录
import os
WORK_DIR = "/content/drive/MyDrive/SAM3_Tracking"
os.makedirs(f"{WORK_DIR}/videos", exist_ok=True)
os.makedirs(f"{WORK_DIR}/outputs", exist_ok=True)

print(f"✅ 请将视频上传到: {WORK_DIR}/videos/")
print(f"✅ 输出将保存到: {WORK_DIR}/outputs/")


In [ ]:
# 列出已上传的视频
videos = [f for f in os.listdir(f"{WORK_DIR}/videos") if f.endswith('.mp4')]
print(f"找到 {len(videos)} 个视频:")
for v in videos[:10]:
    print(f"  - {v}")
if len(videos) > 10:
    print(f"  ... 还有 {len(videos) - 10} 个")


## 3️⃣ 加载 SAM3 模型


In [ ]:
import torch
from sam3 import build_sam3_video_predictor

# 构建视频预测器（会自动下载模型）
print("正在加载 SAM3 模型...")
predictor = build_sam3_video_predictor(device="cuda")
print("✅ 模型加载完成！")


## 4️⃣ 定义追踪函数


In [ ]:
import json
import cv2
import numpy as np
from collections import defaultdict
from PIL import Image

def track_video_with_sam3(predictor, video_path, text_prompts, output_json, confidence_threshold=0.3):
    """
    使用 SAM3 追踪视频中的目标
    
    Args:
        predictor: SAM3 视频预测器
        video_path: 输入视频路径
        text_prompts: 文本提示列表，如 ["car", "person", "traffic sign"]
        output_json: 输出 JSON 路径
        confidence_threshold: 置信度阈值
    """
    print(f"\n🎬 处理视频: {video_path}")
    print(f"🏷️ 检测目标: {text_prompts}")
    
    # 获取视频信息
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    
    print(f"   视频信息: {width}x{height}, {fps:.1f}fps, {frame_count}帧, {frame_count/fps:.1f}秒")
    
    # 初始化视频状态
    state = predictor.init_state(video_path=video_path)
    
    all_tracks = {}  # {(prompt, obj_id): [(frame_idx, box, score), ...]}
    
    # 对每个文本提示进行追踪
    for prompt in text_prompts:
        print(f"\n   📍 追踪: '{prompt}'")
        
        # 重置状态
        predictor.reset_state(state)
        
        # 在第一帧添加文本提示
        frame_idx, obj_ids, masks = predictor.add_new_text(
            inference_state=state,
            frame_idx=0,
            text=prompt
        )
        
        print(f"      第一帧检测到 {len(obj_ids)} 个目标")
        
        # 传播追踪到所有帧
        for frame_idx, obj_ids, video_res_masks in predictor.propagate_in_video(state):
            # 从 mask 计算 bounding box
            for i, obj_id in enumerate(obj_ids):
                if i < len(video_res_masks):
                    mask = video_res_masks[i].cpu().numpy().squeeze()
                    
                    # 计算 bounding box
                    if mask.any():
                        rows = np.any(mask, axis=1)
                        cols = np.any(mask, axis=0)
                        y1, y2 = np.where(rows)[0][[0, -1]]
                        x1, x2 = np.where(cols)[0][[0, -1]]
                        
                        key = (prompt, obj_id)
                        if key not in all_tracks:
                            all_tracks[key] = []
                        
                        all_tracks[key].append({
                            "frame": frame_idx,
                            "x": float(x1) / width * 100,
                            "y": float(y1) / height * 100,
                            "width": float(x2 - x1) / width * 100,
                            "height": float(y2 - y1) / height * 100,
                            "time": frame_idx / fps
                        })
        
        print(f"      追踪完成")
    
    # 转换为 Label Studio 格式
    ls_results = []
    
    for (prompt, obj_id), track in all_tracks.items():
        if track:
            sequence = sorted(track, key=lambda x: x["frame"])
            for item in sequence:
                item["enabled"] = True
                item["rotation"] = 0
            
            ls_results.append({
                "from_name": "box",
                "to_name": "video",
                "type": "videorectangle",
                "value": {
                    "sequence": sequence,
                    "labels": [prompt]
                },
                "id": f"{prompt}_{obj_id}"
            })
    
    # 保存 JSON
    output_data = [{
        "data": {
            "video": f"/data/local-files/?d={os.path.basename(video_path)}"
        },
        "predictions": [{
            "result": ls_results,
            "model_version": "SAM3-VideoPredictor"
        }]
    }]
    
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ JSON 已保存: {output_json}")
    print(f"   共追踪到 {len(ls_results)} 个目标轨迹")
    
    return output_data


## 5️⃣ 运行追踪（单个视频测试）
